In [0]:
import spacy_udpipe
from google.colab import files
import string
from wiki_ru_wordnet import WikiWordnet

wikiwordnet = WikiWordnet()

In [0]:
from google.colab import files


### Загрузка текстов

In [0]:
text = uploaded['decript_clean.txt'].decode('UTF-8')

In [0]:
text[:100]

'Библиотека основана в 1973 году. Расположена в деревне Аксёново Дмитриевского сельского поселения Га'

### Загрузка русского корпуса

In [31]:
spacy_udpipe.download("ru")
nlp = spacy_udpipe.load("ru")

Successfully downloaded the pretrained UDPipe model for the 'ru' language


### Тестирование

In [0]:
# is_digit

In [0]:
examples = ['Библиотека основана в 1973 году.', 'Малышевская сельская библиотека была основана в 1936 году', 'Библиотека № 1 была открыта в феврале 1968 года', 'Дата основания районной библиотеки – 1983 год.']

In [0]:
# is_digit

def view_structure(text):
  
  doc = nlp(text)

  for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.head)

In [110]:
for ex in examples:
  view_structure(ex)
  print()

Библиотека библиотека NOUN nsubj:pass основана
основана основать VERB ROOT основана
в в ADP case году
1973 1973 ADJ amod году
году год NOUN obl основана
. . PUNCT punct основана

Малышевская Малышевский ADJ amod библиотека
сельская сельский ADJ amod библиотека
библиотека библиотека NOUN nsubj:pass основана
была быть AUX aux:pass основана
основана основать VERB ROOT основана
в в ADP case году
1936 1936 ADJ amod году
году год NOUN obl основана

Библиотека библиотека NOUN nsubj:pass открыта
№ № SYM nmod Библиотека
1 1 NUM nummod №
была быть AUX aux:pass открыта
открыта открыть VERB ROOT открыта
в в ADP case феврале
феврале февраль NOUN obl открыта
1968 1968 ADJ amod года
года год NOUN nmod феврале

Дата дата NOUN ROOT Дата
основания основание NOUN nmod Дата
районной районный ADJ amod библиотеки
библиотеки библиотека NOUN nmod основания
– – PUNCT punct год
1983 1983 ADJ amod год
год год NOUN nmod Дата
. . PUNCT punct Дата



***Схема:*** основана -> году -> 1936

### Тестирование Wordnet для русского

In [0]:
def get_syn(word):
  synsets = wikiwordnet.get_synsets(word)
  for index, item in enumerate(synsets):
    for w in item.get_words():
      print(index, w.lemma())

In [88]:
wordnet_ex = ['основывать', 'основать', 'открыть', 'находиться']


for word in wordnet_ex:
  get_syn(word)
  print()

0 основывать
1 основывать

0 основать
1 основать
1 учредить

0 открыть
1 открыть
2 открыть

0 находиться
1 находиться
2 располагаться
2 находиться
3 обретаться
3 находиться



### Дата основания

* ('была', 'открыта', 'в')
* ('библиотека', 'была', 'открыта')
* ('библиотека', 'основана', 'в')
* ('была', 'основана', 'в')
* ('библиотека', 'была', 'основана')
* Дата рождения Всеволожской городской библиотеки – 19 мая 1908 года. 
* Свою историю библиотека ведет с 1932 года
* В 1945 году в здании клуба была организована сельская библиотека
* Официальная дата рождения библиотеки 18 июля 1954 года
* Дата основания районной библиотеки – 1983 год.
* Пищалкинская сельская библиотека работает в поселке Пищалкино с 1987 года. 

---> открыта, основана